In [1]:
import numpy as np
import  tkinter as tk
from tkinter import filedialog
import cv2
import cv2
import os

In [7]:
def load_images_from_folder(folder):
    images = []
    names = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
            names.append(filename)
    return images, names

In [8]:
# define matrix
def getMainMatrix(image_1, image_2):
    
    # define the main matrix with the required size
    mainMatrix = np.zeros((len(image_1) + 1, len(image_2) + 1))
    
    # declare the matchscore, gapscore, and mismatchscore
    matchScore = 1
    gapScore = -2
    mismatchScore = -1
    
    # fix the first element of the matrix as 0
    mainMatrix[0][0] = 0
    
    for i in range(len(image_1) + 1):
        for j in range(len(image_2) + 1):
            
            # fill the first row of the matrix by adding the gapscores
            if i == 0:
                mainMatrix[i][j] = gapScore*j
                
            # fill the second row of the matrix by adding the gapscores
            elif j == 0:
                mainMatrix[i][j] = gapScore*i
                
            # fill the remaining elements of the matrix with either the matchscore or the mismatch score
            else:
                elements = []
                elements.append(mainMatrix[i - 1][j] + gapScore)
                elements.append(mainMatrix[i][j - 1] + gapScore)
                
                # add matchscore if the elements are same
                if(image_1[i - 1] == image_2[j - 1]):
                    elements.append(mainMatrix[i - 1][j - 1] + matchScore)
                    
                # add mismatchscore if the elements are different
                else:
                    elements.append(mainMatrix[i - 1][j - 1] + mismatchScore)
                mainMatrix[i][j] = max(elements)
    
    return mainMatrix

In [9]:
# needlemanwunsch algorithm
def needlemanWunsch(image_1, image_2):
    
    string1 = ""
    string2 = ""

    i = len(image_1)
    j = len(image_2)
    same = 0
    
    # obtain the main matrix for sequence 1 and sequence 2
    mainMatrix = getMainMatrix(image_1, image_2)
#     print(mainMatrix)
    
    # declare empty arrays for storing insertions, deletions, and substitutions
    insertions = []
    deletions = []
    substitutions = []

    while (i > 0 and j > 0):
        
        # when the elements of both the sequences are equal the elements are added to the strings 
        if(image_1[i - 1] == image_2[j - 1]):
            string1 = str(image_1[i - 1]) + string1
            string2 = str(image_2[j - 1]) + string2
            i = i - 1
            j = j - 1
            same = same + 1
        
        else:
            # surrounding elements are added to the array
            elements = []
            elements.append(mainMatrix[i - 1][j])
            elements.append(mainMatrix[i][j - 1])       
            elements.append(mainMatrix[i - 1][j - 1])
            
            # maximum element is identified
            final = max(elements)
            
            # index of the maximum element is identified
            index = elements.index(final)
            
            # check the index value and append the i and j values to the corresponding arrays
            if (index == 0):
                insertions.append(i - 1)
                i = i - 1
                string1 = "-" + string1
            elif (index == 1):
                deletions.append(j - 1)
                j = j - 1
                string2 = "-" + string2
            else:
                substitutions.append(j - 1)
                i = i - 1
                j = j - 1
                string1 = "*" + string1
                
    return same

In [10]:
def getMatch(image):
    x, y, z = image.shape
    flattened = image.flatten()
    folder = "D:\Transfer Files\End Semester\DAA\Face Dataset"
    images, names = load_images_from_folder(folder)
    scores = []
    for i in images:
        resized_img = cv2.resize(i, dim, interpolation = cv2.INTER_AREA)
        flat_img = resized_img.flatten()
        scores.append(needlemanWunsch(flat_img, flattened))
    maxScore = max(scores)
    index = scores.index(maxScore)
    image = images[index]
    name = names[index]
    return name, index

In [12]:
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()
scale_percent = 12.5
img = cv2.imread(file_path)
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
dim = (width, height)
resized_img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
print(getMatch(resized_img))

('sbains.6.jpg', 10)
